In [1]:
import numpy as np
from deap import base
from deap import creator
from deap import tools
import random
import pandas as pd
from deap import algorithms


# Reading From Excel File

In [2]:
df = pd.read_excel(r"C:\Users\mcf\OneDrive\Desktop\Availability (1).xlsx")

In [3]:
samples = np.asarray(df)

In [4]:
df

,country,Fertilizers,Import&Product,Export,Availability,Use,x
0,Brazil,"Ammonia, anhydrous",24144572.05,399156.28,23745415.77,2571549.00,5931.614778
1,Brazil,Ammonium nitrate (AN),21546416.92,214625.65,21331791.27,13821890.37,31882.001541
2,Brazil,Ammonium sulphate,30988444.53,53840.25,30934604.28,22805149.59,52603.066217
3,Brazil,Calcium ammonium nitrate (CAN) and other mixtu...,2020987.02,10844.03,2010142.99,1164236.84,2685.464848
4,Brazil,Diammonium phosphate (DAP),6948831.81,148854.13,6799977.68,4846157.53,11178.297447
...,...,...,...,...,...,...,...
110,Mexico,Sodium nitrate,32779.02,1030.91,31748.11,0.00,0.000000
111,Mexico,Superphosphates above 35%,622825.23,2467787.77,-1844962.54,811919.38,1699.959390
112,Mexico,"Superphosphates, other",0.00,0.00,0.00,21682383.00,45397.574551
113,Mexico,Urea,21917270.86,303460.75,21613810.11,8931655.72,18700.689238


#### In this part we will convert the fertalize names and country names to numbers

In [5]:
country = pd.factorize(df['country'])[0]
fert = pd.factorize(df['Fertilizers'])[0]

In [6]:
samples[:,0] = country
samples[:,1] = fert

In [7]:
samples.shape

(115, 7)

# EC Code
#### in this part we will use availability and production as a fitness functions per country

In [8]:
creator.create("Fitness", base.Fitness, weights = (1.0,))
creator.create("Individual", np.ndarray, fitness = creator.Fitness)

In [9]:
IND_SIZE = 22
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [47]:
def evaluation (individual):
    result1 = 0
    result2 = 0
    for i in range (22):
        temp = individual[i]
        for j in range (115):
            if samples [j,0] == 4 and samples [j,1] == individual[i]:
                result1 += samples[j,4] * (23 - i)
                result2 += samples[j,6] * (23 - i)
    return (result1 + result2),

In [48]:
toolbox.register("evaluate", evaluation)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes)# indpb = random.randint(0,10)/10)
toolbox.register("select", tools.selTournament, tournsize = 10)

In [49]:

def main ():
    res= np.zeros (6000).reshape(2000,3)
    pop = toolbox.population(n = 50)
    fitnesses = list (map (toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    CXPB, MUTPB = 0.8, 0.1
    fits = [ind.fitness.values[0] for ind in pop]
    g = 0
    while g < 500:
        
        print ('--Generation %i---' %g)
        offspring = toolbox.select(pop, len(pop))
        #toolbox.clone() method ensure that we don’t use a reference to the individuals but an completely independent instance.
        offspring = list(map(toolbox.clone, offspring))
                # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant, random.random())
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        res[g,0] , res[g,1,] , res[g,2] = min(fits) , max(fits), mean
        g = g + 1
    ind = max(fits)
    print (ind)
    # you can choose haow many best individuals wanna see by k
    winner = tools.selBest(pop, k = 1)
    return res, winner


In [50]:
res5, winner = main()

--Generation 0---
  Min 779946095.5995708
  Max 1424059873.0438015
  Avg 1110909610.4769359
  Std 139971684.3092986
--Generation 1---
  Min 677206300.7879863
  Max 1424059873.0438015
  Avg 1242184213.298373
  Std 137281944.53814134
--Generation 2---
  Min 752660246.0563568
  Max 1429396849.2891693
  Avg 1325408798.686925
  Std 133868979.70472425
--Generation 3---
  Min 717940247.8476528
  Max 1436734653.9817073
  Avg 1344706230.2222831
  Std 162636317.46998543
--Generation 4---
  Min 879098831.3598323
  Max 1462965214.7964263
  Avg 1400617195.0412834
  Std 120621767.46384974
--Generation 5---
  Min 981022835.7194196
  Max 1463320564.4862301
  Avg 1394122610.5312552
  Std 124646834.49387509
--Generation 6---
  Min 1071143960.0852959
  Max 1463430956.2364264
  Avg 1439244523.1706436
  Std 70154652.95078067
--Generation 7---
  Min 651604465.6128776
  Max 1463855769.8421466
  Avg 1425069542.7457974
  Std 136337941.76766917
--Generation 8---
  Min 1008117610.1953508
  Max 1464369204.2900202

In [54]:
import matplotlib.pyplot as plt
%matplotlib

plt.plot (res2[0:500,2], color = 'yellow', label = 'Brazil')
plt.plot (res[0:500,2], color = 'coral', label = 'USA')
plt.plot (res3[0:500,2], color = 'lightblue', label = 'Turkey')
plt.plot (res5[0:500,2], color = 'r', label = 'Mexico')
plt.plot (res4[0:500,2], color = 'g', label = 'India')
plt.ylabel ('Mean Fitenss')
plt.xlabel ('Generation = 500')
plt.legend()

plt.show()

Using matplotlib backend: Qt5Agg


In [55]:
winner

[Individual([21,  2, 13,  0,  7,  4, 10, 15,  8,  5,  1, 17,  6, 12,  9,
             16, 18, 11, 20, 14,  3, 19])]